In [5]:
#Second part of project
import cv2
import numpy as np
import time

# Initalize camera
cap = cv2.VideoCapture(0)

# define range of green color in HSV
# Instead of a green colour object we will be using two different objects like 
# 1. Sticker
# 2. Object by darknet

#Testing for a green colour object
lower_green = np.array([45,50,50])
upper_green = np.array([75,200,200])

# Create empty points array
points = []

#Frame Count
frame_count=0

# Get default camera window size
ret, frame = cap.read()

#Height and Width of the frame
Height, Width = frame.shape[:2]

while True:
    # Capture webcame frame
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blank_image = np.zeros((frame.shape[0], frame.shape[1], 3))
    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv_img, lower_green, upper_green)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #Defining our ranges for left and right
    fifteen_per = 0.40*(Width/2)
    centre_region_begin = Width/2-fifteen_per
    centre_region_end = Width/2+fifteen_per
    centre_region_begin_part2 = Width/2-2*fifteen_per
    centre_region_end_part2 = Width/2+2*fifteen_per
    
    
    if frame_count>=0:
        #Conditions used for Green colour object 
        if len(contours) > 0:
            #Conditions used for Green colour object
            c = max(contours, key=cv2.contourArea)
            #Conditions used for Green colour object
            (x, y), radius = cv2.minEnclosingCircle(c)
            # Allow only countors that have a larger than 15 pixel radius
            if radius > 9:
                #Drawing the Circle for the object
                cv2.circle(blank_image, (int(x), int(y)), int(radius),(0, 0, 255), 2)
                #Drawing the Circle for the centre
                cv2.circle(blank_image, (int(x), int(y)), 5, (0, 255, 0), -1)
                points.append((int(x), int(y)))
                #Text to be displayed
                text = ""
                if int(x)>centre_region_begin and int(x)<=centre_region_end:
                    text = "NO CHANGE"
                elif int(x)>centre_region_end and int(x)<=centre_region_end_part2:
                    text = "RIGHT"
                elif int(x)>centre_region_end_part2:
                    text = "MORE RIGHT"
                elif int(x)>=centre_region_begin_part2 and int(x)<=centre_region_begin:
                    text = "LEFT"
                elif int(x)<centre_region_begin_part2:
                    text = "MORE LEFT"
                cv2.putText(blank_image, text, (int(x)+20, int(y)+20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255),2)
            else:
                if(len(points)!=0):
                    (x,y) = points[-1]
                    if int(x)>centre_region_begin and int(x)<=centre_region_end:
                        text = "NO CHANGE"
                    elif int(x)>centre_region_end and int(x)<=centre_region_end_part2:
                        text = "RIGHT"
                    elif int(x)>centre_region_end_part2:
                        text = "MORE RIGHT"
                    elif int(x)>=centre_region_begin_part2 and int(x)<=centre_region_begin:
                        text = "LEFT"
                    elif int(x)<centre_region_begin_part2:
                        text = "MORE LEFT"
                    cv2.putText(blank_image, text, (int(x)+20, int(y)+20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255),2)
        else:
            if(len(points)!=0):
                    (x,y) = points[-1]
                    if int(x)>centre_region_begin and int(x)<=centre_region_end:
                        text = "NO CHANGE"
                    elif int(x)>centre_region_end and int(x)<=centre_region_end_part2:
                        text = "RIGHT"
                    elif int(x)>centre_region_end_part2:
                        text = "MORE RIGHT"
                    elif int(x)>=centre_region_begin_part2 and int(x)<=centre_region_begin:
                        text = "LEFT"
                    elif int(x)<centre_region_begin_part2:
                        text = "MORE LEFT"
                    cv2.putText(blank_image, text, (int(x)+20, int(y)+20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255),2)
    frame_count+=1
    cv2.imshow("Object Tracker", blank_image)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

# Release camera and close any open windows
cap.release()
cv2.destroyAllWindows()